In [ ]:
import os
import skspeech

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Some plotting niceties
plt.rc('figure', figsize=(10, 6))
sns.set_style('white')
sns.set_style('ticks')

import nengo
from nengo import spa
from nengo.spa import Vocabulary
import nengo_gui.ipython
from nengo.dists import Choice, Uniform

import phd

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

In [ ]:
# Create our semantic pointers
dimensions = 96
rng = np.random.RandomState(0)
vocab = Vocabulary(dimensions=dimensions, rng=rng, unitary=['NEXTPOS', 'POS1'])
vocab.parse('NEXTPOS')
vocab.parse('POS1')
for i in xrange(2, 9):
    vocab.add('POS%d' % i, vocab.parse('POS%d * NEXTPOS' % (i-1)))

syllables = ('PAT', 'DAS', 'KAP')
for syll in syllables:
    vocab.parse(syll)

just_syllables = vocab.create_subset(syllables)

In [ ]:
difference_gain = 15
neurons_per_d = 50

with spa.SPA() as model:
    model.sequence = spa.Buffer(dimensions=dimensions, neurons_per_dimension=neurons_per_d)

    model.syll_idx1 = nengo.networks.InputGatedMemory(
        neurons_per_d, dimensions=dimensions, difference_gain=difference_gain)
    model.syll_idx2 = nengo.networks.InputGatedMemory(
        neurons_per_d, dimensions=dimensions, difference_gain=difference_gain)
    nengo.Connection(model.syll_idx1.output, model.syll_idx2.input)
    nengo.Connection(model.syll_idx2.output, model.syll_idx1.input,
                     transform=vocab['NEXTPOS'].get_convolution_matrix())

    model.gate = nengo.Ensemble(100, dimensions=1,
                                encoders=nengo.dists.Choice([[1]]),
                                intercepts=nengo.dists.Uniform(0.2, 1))
    nengo.Connection(model.gate, model.syll_idx1.gate)
    # bias so that model.syll_idx2.gate gets 1 - gate
    nengo.Connection(nengo.Node(1, label='gate bias'), model.syll_idx2.gate)
    nengo.Connection(model.gate, model.syll_idx2.gate, transform=[-1])
    
    # syll = sequence * ~syll_idx
    model.bind = nengo.networks.CircularConvolution(
        neurons_per_d, dimensions, invert_a=False, invert_b=True)
    nengo.Connection(model.sequence.state.output, model.bind.A)
    nengo.Connection(model.syll_idx1.output, model.bind.B)
    
    # For quick testing
    nengo.Connection(nengo.Node(lambda t: 1 if (t % 0.2) > 0.1 else 0), model.gate)
    init_idx = nengo.Node(lambda t: vocab.parse('POS1').v if t < 0.2 else vocab.parse('0').v)
    nengo.Connection(init_idx, model.syll_idx1.input)
    nengo.Connection(nengo.Node(vocab.parse('PAT*POS1 + DAS*POS2 + KAP*POS3').v),
                     model.sequence.state.input)

    p_gate = nengo.Probe(model.gate, synapse=0.01)
    p_syllidx = nengo.Probe(model.syll_idx1.output, synapse=0.01)

In [ ]:
with model:
    model.syllable = spa.AssociativeMemory(just_syllables, wta_output=True)  # modify them here...
    nengo.Connection(model.bind.output, model.syllable.input)
    p_syll = nengo.Probe(model.syllable.output, synapse=0.01)

In [ ]:
sim = nengo.Simulator(model)
sim.run(.6)

In [ ]:
plt.figure(figsize=(12, 12))
    
plt.subplot(3, 1, 1)
plt.plot(sim.trange(),
         spa.similarity(sim.data[p_syllidx],
                        vocab.create_subset(['POS1', 'POS2', 'POS3'])))
plt.legend(['POS1', 'POS2', 'POS3'], loc='best')
plt.ylim([-0.2, 1.2])

plt.subplot(3, 1, 2)
plt.plot(sim.trange(), sim.data[p_gate])
plt.ylim([-0.1, 1.1])

plt.subplot(3, 1, 3)
plt.plot(sim.trange(), 
         spa.similarity(sim.data[p_syll],
                        vocab.create_subset(syllables)))
plt.legend(syllables, loc='best')

In [ ]:
def get_traj(path, dt=0.001):
    gs = skspeech.vtl.parse_ges(path)
    wavfile = os.path.basename(path)[:-4] + '.wav'
    if not os.path.exists(wavfile):
        gs.synthesize(wavfile=wavfile)
    traj = gs.trajectory(dt=dt)
    return traj

def rescale(val, old_min, old_max, new_min, new_max):
    old_range = old_max - old_min
    new_range = new_max - new_min
    return (((val - old_min) * new_range) / old_range) + new_min

def traj2func(traj, dt=0.001):
    traj_ix = np.unique(np.nonzero(traj)[1])
    traj = traj[:, traj_ix]
    t_end = traj.shape[0] * dt
    def _trajf(x):
        # x goes from 0 to 1; normalize by t_end
        # But give a bit of 0 at the start...
        if x < 0.1:
            return np.zeros(traj.shape[1])
        ix = min(int(rescale(x, 0.1, 1., 0., t_end-0.1) / dt), traj.shape[0]-1)
        return traj[ix]
    _trajf.traj = traj
    _trajf.ix = traj_ix
    return _trajf, traj_ix

dt = 0.001
pat = get_traj(phd.ges_path('ges-de-cvc', 'pat.ges'), dt)
das = get_traj(phd.ges_path('ges-de-cvc', 'das.ges'), dt)
kap = get_traj(phd.ges_path('ges-de-cvc', 'kap.ges'), dt)
pat_f, pat_ix = traj2func(pat, dt)
das_f, das_ix = traj2func(das, dt)
kap_f, kap_ix = traj2func(kap, dt)

In [ ]:
from phd.networks import dmp
dmp = reload(dmp)

def kick_f(x):
    # We'll do this as a function of x, like the forcing_f
    return 1 if x > 0.8 else 0

# dz = 0.6 * np.pi

with nengo.Network() as model:
    pat_dmp = dmp.RhythmicDMP(200, freq=2.5, forcing_f=pat_f) # , deadzone=dz)
    das_dmp = dmp.RhythmicDMP(200, freq=5.3, forcing_f=das_f) # , deadzone=dz)
    kap_dmp = dmp.RhythmicDMP(200, freq=2.5, forcing_f=kap_f) # , deadzone=dz)

    speech = nengo.networks.EnsembleArray(60, pat.shape[1])
    nengo.Connection(pat_dmp.output, speech.input[pat_ix])
    nengo.Connection(das_dmp.output, speech.input[das_ix])
    nengo.Connection(kap_dmp.output, speech.input[kap_ix])

    # disinhibit one to let it through
    # switch at 1 second
    nengo.Connection(nengo.Node(lambda t: 1 if t < 0.93 else 0), pat_dmp.disinhibit)
    nengo.Connection(nengo.Node(lambda t: 1 if t > 0.85 else 0), das_dmp.disinhibit)
    
    # Kick is provided by the oscillators also
    init_kick = nengo.Node(output=lambda t: 0.8 if t < 0.08 else 0.0)
    kick = nengo.Node(size_in=1)
    nengo.Connection(init_kick, kick)
    nengo.Connection(kick, pat_dmp.kick)
    nengo.Connection(pat_dmp.osc, kick, function=dmp.radial_f(kick_f)) # , dz))
    nengo.Connection(kick, das_dmp.kick)
    nengo.Connection(das_dmp.osc, kick, function=dmp.radial_f(kick_f)) # , dz))
    nengo.Connection(kick, kap_dmp.kick)
    nengo.Connection(kap_dmp.osc, kick, function=dmp.radial_f(kick_f)) # , dz))

    p_kick = nengo.Probe(kick)
    p_pat = nengo.Probe(pat_dmp.osc, synapse=0.01)
    p_das =nengo.Probe(das_dmp.osc, synapse=0.01)
    p_what = nengo.Probe(das_dmp.output, synapse=0.01)
    p_sa = nengo.Probe(speech.output, synapse=0.01)

In [ ]:
sim = nengo.Simulator(model)
sim.run(1.8)

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(sim.data[p_pat].T[0], sim.data[p_pat].T[1])
plt.subplot(1, 2, 2)
plt.plot(sim.data[p_das].T[0], sim.data[p_das].T[1])
plt.figure()
plt.plot(sim.trange(), sim.data[p_kick], c='k')
plt.plot(sim.trange(), sim.data[p_sa]);